In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if 'google.colab'in sys.modules:
    !pip install scikit-decide[all]

In [3]:
!(rm -rf scikit-maze/; git clone https://github.com/galleon/scikit-maze.git)

Cloning into 'scikit-maze'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 84 (delta 30), reused 70 (delta 16), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [4]:
sys.path.insert(0,'./scikit-maze')

In [27]:
from enum import Enum
from typing import Any, List, NamedTuple, Optional

from skdecide import DeterministicPlanningDomain, Space, Value
from skdecide.builders.domain import UnrestrictedActions, Renderable
from skdecide.utils import rollout, match_solvers, load_registered_solver
from skdecide.hub.space.gym import ListSpace, EnumSpace, MultiDiscreteSpace
from skdecide.hub.solver.lazy_astar import LazyAstar

from utils import Maze

Create a Maze

In [6]:
maze = Maze(25, 25)

In [7]:
from PIL import Image

im_array = maze.get_image().repeat(4, 0).repeat(4, 1)

In [10]:
import io
import ipywidgets as widgets

In [16]:
buff = io.BytesIO()
img = Image.fromarray(im_array)
img.save(buff, format='png')

In [19]:
# widgets.Image(value=buff.getvalue(), format='png', layout=widgets.Layout(width='200px'))
widgets.Image(value=buff.getvalue(), layout=widgets.Layout(width='200px'))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00d\x00\x00\x00d\x08\x00\x00\x00\x00U\x89\xca\x88\x…

## Define Action & State spaces

In [15]:
class State(NamedTuple):
    x: int
    y: int

class Action(Enum):
    up = 0
    down = 1
    left = 2
    right = 3

## Define a base domain

In [21]:
class D(DeterministicPlanningDomain, UnrestrictedActions, Renderable):
    T_state = State  # Type of states
    T_observation = T_state  # Type of observations
    T_event = Action  # Type of events
    T_value = float  # Type of transition values (rewards or costs)
    T_predicate = bool  # Type of logical checks
    T_info = None  # Type of additional information in environment outcome

## Create the maze domain

In [88]:
from math import sqrt

class MazeDomain(D):

    def __init__(self, start, end, maze, image_widget=None):
        self.start = start
        self.end = end
        self.maze = maze
        self.image_widget = image_widget
        #if self.image_widget:
        #    self.image_widget.layout = widgets.Layout(width='200px')

    def _get_next_state(self, memory: D.T_state, action: D.T_event) -> D.T_state:
        # Move agent according to action (except if bumping into a wall)
        next_x, next_y = memory.x, memory.y
        if action == Action.up:
            next_x -= 1
        if action == Action.down:
            next_x += 1
        if action == Action.left:
            next_y -= 1
        if action == Action.right:
            next_y += 1
        return State(next_x, next_y) if self.maze.is_an_empty_cell(next_x, next_y) else memory

    def _get_transition_value(self, memory: D.T_state, action: D.T_event, next_state: Optional[D.T_state] = None) -> \
            Value[D.T_value]:
        # Set cost to 1 when moving (energy cost) and to 2 when bumping into a wall (damage cost)
        return Value(cost=1 if next_state != memory else 2)

    def _get_initial_state_(self) -> D.T_state:
        # Set the start position as initial state
        return self.start

    def _get_goals_(self) -> Space[D.T_observation]:
        # Set the end position as goal
        return ListSpace([self.end])

    def _is_terminal(self, state: D.T_state) -> D.T_agent[D.T_predicate]:
        # Stop an episode only when goal reached
        return self._is_goal(state)

    def _get_action_space_(self) -> Space[D.T_event]:
        # Define action space
        return EnumSpace(Action)

    def _get_observation_space_(self) -> Space[D.T_observation]:
        # Define observation space
        num_rows = self.maze.height
        num_cols = self.maze.width
        return MultiDiscreteSpace([num_rows, num_cols])

    def _render_from(self, memory: D.T_state, **kwargs: Any) -> Any:
        #  display maze in an image widget
        #with io.BytesIO() as buff:
        #    img = Image.fromarray(maze.get_image().repeat(4, 0).repeat(4, 1))
        #    img.save(buff, format='png')
        #    self.image_widget.value = buff.getvalue()
        buff = io.BytesIO()
        maze_ = maze.get_image(memory.x, memory.y).repeat(4, 0).repeat(4, 1)
        img = Image.fromarray(maze_)
        img.save(buff, format='png')
        self.image_widget.value = buff.getvalue()
        
        
    def heuristic(self, s: D.T_state) -> Value:
        return Value(cost=sqrt((self.end.x - s.x)**2 + (self.end.y - s.y)**2))
    
    def state_features(self, s: D.T_state) -> List[float]:
        return [s.x, s.y]

## Define a Maze

In [89]:
height, width = 25, 25

domain_factory = lambda: MazeDomain(State(1, 1), State(height-2, width-2), maze=Maze(width, height))

## Let solve with A*

In [90]:
solver = LazyAstar()
solver._initialize()
MazeDomain.solve_with(solver, domain_factory)

## Rollout

In [91]:
max_steps = (height-2)*(width-2)

output = widgets.Image(format='png')

#output.value = buff.getvalue()

output

Image(value=b'')

In [92]:
domain = MazeDomain(State(1, 1), State(height-2, width-2), maze=Maze(width, height), image_widget=output)
rollout(domain, solver, max_steps=max_steps, max_framerate=80, verbose=False)

2021-09-14 20:34:16,793 | skdecide.utils | INFO | The goal was not reached in episode 1.
